<a href="https://colab.research.google.com/github/yunwilliamyu/interactive-fiction-class.github.io/blob/master/homeworks/language-model/hw4_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4 - Finetune GPT-2
This colab demonstrates how to fine-tune GPT-2 on a dataset of presidential speeches. We use the [Hugging Face Transformer](https://github.com/huggingface/transformers) library in order to do this.

**IMPORTANT: Make sure that you have GPU set as your Hardware Accelerator in `Runtime > Change runtime type` before running this Colab.**

## Setup

### Install HuggingFace Transfomers library.

In [1]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples')

!pip install dict_to_obj


Cloning into 'transformers'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 19942 (delta 17), reused 24 (delta 7), pack-reused 19901
Receiving objects: 100% (19942/19942), 11.99 MiB | 7.51 MiB/s, done.
Resolving deltas: 100% (14479/14479), done.
Processing /content/transformers
     |████████████████████████████████| 3.8MB 2.8MB/s 
     |████████████████████████████████| 1.0MB 34.7MB/s 
     |████████████████████████████████| 870kB 27.8MB/s 
  Created wheel for transformers: filename=transformers-2.5.0-cp36-none-any.whl size=497322 sha256=37cc1bc79ca37967939a72fec34e976ef45d817c7803cd62d15af696352e646c
  Stored in directory: /tmp/pip-ephem-wheel-cache-o4jia_nu/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=ccfe0fb85df38706cfb17eca0d485757627d37f058e0a827744edbe762ddd536


In [2]:
import torch
import run_language_modeling
import run_generation
from dict_to_obj import DictToObj
import collections
import random
import numpy as np

### Mount your Google Drive
We will be saving trained checkpoints on your Google Drive so that they can be accessed even if the Colab session dies. Make sure to login with your UPenn credentials, as you will be saving several gigabytes of data, and Penn gives you unlimited Drive storage.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Download presidential speech data.
It's already been split into a train, valid, and test set for you.

In [4]:
# Download the train and test set.
!wget -nc -O /content/presidential_speeches_test.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_test.txt
!wget -nc -O /content/presidential_speeches_valid.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_valid.txt
!wget -nc -O /content/presidential_speeches_train.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_train.txt

--2020-02-22 07:26:11--  https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622537 (608K) [text/plain]
Saving to: ‘/content/presidential_speeches_test.txt’

/content/presidenti 100%[===================>] 607.95K  --.-KB/s    in 0.06s   

2020-02-22 07:26:13 (9.54 MB/s) - ‘/content/presidential_speeches_test.txt’ saved [622537/622537]

--2020-02-22 07:26:14--  https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.13

## Finetune and Eval
The Hugging Face library provides a script [run_language_modeling.py](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) which contains all of the code for training and evaluating a language model.

We will be calling this script directly from the command line in order to launch training. We will also use functions from this script to conduct evaluation and generate samples at inference time.

### Launch fine-tuninng
We will be calling `run_language_modeling.py` from the command line to launch fine-tuning, **Running fine-tuning may take several hours.** Every `save_steps` steps, a checkpoint is saved to disk. The checkpoint contains all the learned weights for your model, and you can  always reload the model from a saved checkpoint, even if your Colab has crashed.

Below is an explanation of some of the arguments you might want to modify in the command below. 

* `--line_by_line`: Add `--line_by_line` if distinct lines of the text should be treated as distinct training examples. For example, if your dataset contains one story/tweet/article per line, this should be set.
* `--num_train_epochs`: The number of times to iterate over the train set. Increasing the number of epochs may result in better performance, but making this number too high will cause the model to overfit on the train set.
* `--block_size`: Your training text is truncated into blocks of this length. At test time, you will only want to generate sequences that are at most this length.
* `--gradient_accumulation_steps`: Update the model weights every this many steps. You shold set this to >1 when the batch size is very small to improve training stability.
* `--output_dir`: This is the where checkpoints will get saved. When you finetune on your own dataset, you should change this path. We recommend saving checkpoints to your Google Drive (`/content/drive/My Drive/`) so you can access them even if the Colab session dies.
* `--model_name_or_path` The path to the model weights to use when starting fine-tuning. You can set this to `gpt2-medium` to initialize with GPT-2's 355 million parameter model, or `gpt2` to initialize with their smaller 124 million parameter model. You can also set this to one of your own checkpoints to restart your training job if it crashes.

**I am getting out-of memory errors. What do I do?**

The number of trainable paramters in the model is a function of the `block_size` and the `batch_size`. If you are getting out-of-memory errors, then try drecreasing these value.

**Oh no! My computer went to sleep and the Colab disconnected.**

The train job might have still completed. Check the `output_dir` in your Google Drive to see if checkpoint files have been created there.

**Training is taking foreverrrrrr.**

Try decreasing `num_train_epochs` or changing `model_name_or_path` to `gpt2` instead of `gpt2-medium`.
If your evaluation set is very large, you might also want to remove the `evaluate_during_training` flag or increase `logging_steps`.

In [0]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/presidential_speeches3' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=5 \
    --save_steps=5 \
    --train_data_file=/content/presidential_speeches_train.txt \
    --do_eval \
    --eval_data_file=/content/presidential_speeches_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5

02/22/2020 07:52:28 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
02/22/2020 07:52:28 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-config.json from cache at /root/.cache/torch/transformers/98aa65385e18b0efd17acd8bf64dcdf21406bb0c99c801c2d3c9f6bfd1f48f29.266bb9683aedfcb1f7006ad2e6894fce82b8dbbae8125f4fc8570b818005b83d
02/22/2020 07:52:28 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_ids": null,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_leng

### Compute perplexity of a dataset.
This section shows how to compute perplexity of a dataset according to either the pre-trained or your fine-tuned language model. While this is possible to do by calling `run_language_modeling.py` on the command-line as above, we'll instead call the Python functions directly.

#### Look at what checkpoints are available
Run `ls` to look at what checkpoints saved been saved. You'll want to set `CHECKPOINT_PATH` below to one of these in order to evaluate the model weights saved in that checkpoint.

In [9]:
!ls '/content/drive/My Drive/finetuned_model/presidential_speeches'

ls: cannot access '/content/drive/My Drive/finetuned_model/presidential_speeches': No such file or directory
 2017May-CSHL-BoG-reimburse.gsheet
 2018-10-06-Adam-papergame-demo-playtest.gdoc
 2019-02-18-levenshtein-review-response.gdoc
 ADI_Summer_Offsite_2015-2015-08-28.zip
'Advanced linear algebra.gdoc'
 AmericanMythos.gsheet
'Amia 2018 notes.gdoc'
 Applications
 ar-sketching-review.gdoc
'Arts Scholars Share Decoration planning.gdoc'
 backup-2016-05-25
'Beauty and the Beast-Belle-SheetMusicDownload.pdf'
 calls-20180619105128.xml
 Chrombook-Downloads-20150115
'Colab Notebooks'
 combined2.csv
 ComMIT
'ComMIT (1)'
'CORA 2 - Design Doc.gdoc'
'Dan Interview 2016-01-24.gdoc'
'Daphne Google Draft.gdoc'
'Daphne Ippolito - Teams.html'
'Daphne Ippolito - Teams.mhtml'
'Daphne notes.gdoc'
 DaphneScratch
'Diana Wedding speech.gdoc'
 documentation
 encfs-small-2018-12-25.tar.bz2
'FB Election 2016.gdoc'
 FLjtaEmUsoi.png
 Genome2017_AbstractBook.pdf
'Gmail table.gdoc'
 Google_Employment_Application_Y

#### Helper functions

In [0]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config_class, model_class, _ = run_language_modeling.MODEL_CLASSES[args.model_type]
  config = config_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = model_class.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None,
  )
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  _, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

#### Compute it.

In [0]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
# CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_model/checkpoint-1500'
CHECKPOINT_PATH = "gpt2-medium"

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/presidential_speeches_valid.txt",
              "/content/presidential_speeches_test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObj(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

Running on device:  cuda


Exception ignored in: <bound method tqdm.__del__ of Evaluating:  26%|██▋       | 4176/15836 [07:40<14:20, 13.56it/s]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 931, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 1133, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 496, in _decr_instances
    cls.monitor.exit()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 52, in exit
    self.join()
  File "/usr/lib/python3.6/threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


Evaluating: 100%|██████████| 496/496 [00:38<00:00, 14.47it/s]


23.510801315307617 is the perplexity of /content/presidential_speeches_valid.txt according to gpt2-medium


Evaluating: 100%|██████████| 496/496 [00:35<00:00, 14.83it/s]

23.510801315307617 is the perplexity of /content/presidential_speeches_test.txt according to gpt2-medium


### Generate samples
The following code generates text samples that are are continuations of a provided prompt.

In [0]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  _, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [0]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_model/checkpoint-1500'

# You should try out other prompts as well as no prompt at all.
PROMPT = '<title=\"Remarks on Mission to Mars\">'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
  temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=100,  # Number of tokens to generate.
  num_return_sequences=3,  # Number of independently computed samples to generate.
)
args = DictToObj(args)

model = load_model(args)
sequences = generate_samples(args, model, PROMPT)
for sequence in sequences:
  print(sequence)

02/22/2020 06:47:31 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_model/checkpoint-1500/config.json
02/22/2020 06:47:31 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "predict_spec

Running on device:  cuda


02/22/2020 06:47:44 - INFO - transformers.tokenization_utils -   Model name '/content/drive/My Drive/finetuned_model/checkpoint-1500' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming '/content/drive/My Drive/finetuned_model/checkpoint-1500' is a path, a model identifier, or url to a directory containing tokenizer files.
02/22/2020 06:47:44 - INFO - transformers.tokenization_utils -   Didn't find file /content/drive/My Drive/finetuned_model/checkpoint-1500/added_tokens.json. We won't load it.
02/22/2020 06:47:44 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/finetuned_model/checkpoint-1500/vocab.json
02/22/2020 06:47:44 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/finetuned_model/checkpoint-1500/merges.txt
02/22/2020 06:47:44 - INFO - transformers.tokenization_utils -   loading file None
02/22/2020 06:47:44 - INFO - transformers.tokenization_utils -   loading file 

=== GENERATED SEQUENCE 1 ===
=== GENERATED SEQUENCE 2 ===
=== GENERATED SEQUENCE 3 ===
<title="Remarks on Mission to Marks">
<president="polk">
<date="March 10, 1967">
The Senate and the House and the people of the United States:
It is in their interest and in our interest to be here today to hear that the Cuban people know what has been required of them in return for their peaceful conversion to democratic communism. For over five years over 200,000 Cubans have lived and worked under the leadership a government which respects their autonomy and who has taken every possible step to protect them
<title="Remarks on Mission to Marks">
<president="bush">
<date="April 14, 2003">
I'm pleased to be here today to say I'm one of 100th President of the United States. The United States of America is truly a great country, and the American people are even more extraordinary. In more than 60 years, more than 90 percent of your lives have been spent in this great house of the Presidency, in this pla

# TODO: Instructions to give students
* Play around with the presidential speech language model a bit. Compare its outputs to the statistical language model and the LSTM.
* Create a dataset of your own. Find some text online and extract it into a .txt file. Ideally your dataset should contain between 10 and 50 MB of text.
* Break your text file into 2 text files, one for training and one for testing. (We'll need to provide instructions on how to do this)
* Report perplexity of your train set and your test set (we'll need to explain what perplexity is) on GPT-2 without finetuning and GPT-2 with finetuning. Perplexity should be lower after finetuning or something has gone wrong.

# TODO: Analysis questions we could ask students
* Generate 100 unconditioned samples from your finetuned model and 100 unconditioned examples from GPT-without finetuning. Do the samples from your finetuned model have higher word overlap with your test set.
* What happens when you set top-k to 1 vs setting it to some large number?